# CAICT 風機結冰預測 - DL-Logistic Regression

2017/07/12  
http://www.industrial-bigdata.com/competition/competitionAction!showDetail.action?competition.competitionId=1

# **1. Load Data and Modules**

**Load Python modules:**

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import datetime
import pandas as pd
import os
import sys
from PIL import Image
import numpy as np
import lzma
import pickle
from IPython.display import display
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector
import shutil
import csv
import sklearn as sk
from sklearn.metrics import precision_score
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from pylab import *
from IPython.core.interactiveshell import InteractiveShell
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

pd.set_option('display.max_columns', None)
InteractiveShell.ast_node_interactivity = "all"

In [2]:
def timerangeCheck(t, start, end):
    return (t>=start) & (t<=end)

def variable_summaries(var, name):  
    with tf.name_scope('summaries_'+str(name)):  
        mean = tf.reduce_mean(var)  
        tf.summary.scalar('mean', mean)  
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))  
        tf.summary.scalar('stddev', stddev)  
        tf.summary.scalar('max', tf.reduce_max(var))  
        tf.summary.scalar('min', tf.reduce_min(var))  
        tf.summary.histogram('histogram', var)  
        
def updateProgress(msg):
    sys.stdout.write('\r')
    sys.stdout.write(msg)
    sys.stdout.flush()
    
def myscore(true_y, pred_y):
    n,p =sk.metrics.confusion_matrix(true_y, pred_y)
    tn = n[0]
    fp = n[1]
    fn = p[0]
    tp = p[1]
    #print('tn:',tn,'fp:',fp,'fn:',fn,'fp:',fp)
    score = 1- 0.5*(fp/(tn+fp))- 0.5*(fn/(fn+tp)) 
    #print('score',score)
    return score, {'tn':tn,'fp':fp,'fn':fn,'tp':tp}


**Load input data.** 

In [3]:
def load_resample_data(id, remove_unknown=True, split_case=20):
    data_resample_15 = pd.read_csv('data/resample_data_15.csv')
    if remove_unknown:
        data_resample_15 = data_resample_15[data_resample_15['label']!=2]
    
    train_resample_15 = data_resample_15[data_resample_15['event']<split_case]
    valid_resample_15 = data_resample_15[data_resample_15['event']>=split_case]
    train_X = train_resample_15.drop(['label','time','group','timestamp','event'], axis=1).values
    train_y = train_resample_15['label'].values.astype(int)
    valid_X = valid_resample_15.drop(['label','time','group','timestamp','event'], axis=1).values
    valid_y = valid_resample_15['label'].values.astype(int)
    train_Y = np.eye(2)[train_y]
    valid_Y = np.eye(2)[valid_y]
    
    train_abnormal = train_resample_15[train_resample_15['label']==1]
    train_normal = train_resample_15[train_resample_15['label']==0]
    train_desc = '正常:{i} ({j:.2f} percent), 結冰:{k:} ({m:.2f} percent), Total:{n}'.format(i=len(train_normal), j=len(train_normal)/len(train_resample_15),k=len(train_abnormal),m=len(train_abnormal)/len(train_resample_15), n=len(train_resample_15))
    
    valid_abnormal = valid_resample_15[valid_resample_15['label']==1]
    valid_normal = valid_resample_15[valid_resample_15['label']==0]
    valid_desc = '正常:{i} ({j:.2f} percent), 結冰:{k:} ({m:.2f} percent), Total:{n}'.format(i=len(valid_normal), j=len(valid_normal)/len(valid_resample_15),k=len(valid_abnormal),m=len(valid_abnormal)/len(valid_resample_15), n=len(valid_resample_15))
    
    return train_X, train_Y, train_y, valid_X, valid_Y, valid_y, train_desc, valid_desc

def load_raw_data(id,remove_unknown=True):
    data_raw_21 = pd.read_csv('data/goodformat_21.csv')
    data_raw_21 = data_raw_21.dropna()
    
    data_X = data_raw_21.drop(['label','time','group','timestamp'], axis=1).values
    data_y = data_raw_21['label'].values.astype(int)
    data_Y = np.eye(2)[data_y]
    
    data_abnormal = data_raw_21[data_raw_21['label']==1]
    data_normal = data_raw_21[data_raw_21['label']==0]
    data_desc = '正常:{i} ({j:.2f} percent), 結冰:{k:} ({m:.2f} percent), Total:{n}'.format(i=len(data_normal), j=len(data_normal)/len(data_raw_21),k=len(data_abnormal),m=len(data_abnormal)/len(data_raw_21), n=len(data_raw_21))
    
    return data_X, data_Y, data_y, data_desc

def load_test_data():
    test_data = pd.read_csv('data/ice1/test/08/08_data.csv')
    test_timeidx = test_data['time']
    tmp = test_data.drop(['time','group'], axis=1)
    test_X = tmp.values
    return test_X, test_timeidx

    

In [4]:
train_X, train_Y, train_y, valid_X, valid_Y, valid_y, train_desc, valid_desc = load_resample_data(15)
valid_21_X, valid_21_Y, valid_21_y, valid_21_desc= load_raw_data(21)
test_X,test_timeidx = load_test_data()

# 2. Initial Exploration
**Look at your data in as many different ways as possible.**

**数据描述**

** Training Data 風機狀態正常(0)/異常(1)比例 **

In [5]:
print(train_desc)
pd.DataFrame(train_X)[:5]

正常:35339 (0.61 percent), 結冰:22179 (0.39 percent), Total:57518


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25
0,2.568083,1.236890,2.483755,1.079588,0.218807,-0.614620,-0.213382,2.280000,2.244444,2.280000,-0.20,-0.16,-0.24,0.010,-0.139,-0.139,1.225767,0.061109,-0.200349,-0.422974,1.090769,1.061538,0.952308,0.64,2.84,0.16
1,2.405812,1.263480,2.495777,1.034703,1.482811,-0.614620,-0.178499,2.057778,2.031111,2.048889,0.24,0.36,0.12,0.040,-0.120,-0.120,1.225767,0.061109,-0.184568,-0.422974,1.076923,1.046154,0.952308,-0.36,0.28,0.00
2,2.402124,1.286747,2.485770,1.076135,1.320694,-0.591323,0.344757,2.297778,2.271111,2.271111,2.40,2.16,2.20,0.080,-0.100,-0.089,1.225767,0.061109,-0.200349,-0.422974,1.076923,1.046154,0.952308,-0.84,0.20,0.04
3,2.446380,1.263480,2.497769,1.036429,1.292830,-0.591323,0.240106,2.204444,2.186667,2.204444,1.16,0.84,0.68,0.100,-0.069,-0.060,1.225767,-1.209522,-0.200349,-0.422974,1.076923,1.046154,0.936923,0.44,1.88,0.12
4,2.420564,1.286747,2.497769,1.003628,0.370791,-0.614620,-0.318034,2.528889,2.511111,2.502222,1.44,1.48,1.48,0.139,-0.040,-0.050,1.225767,0.061109,-0.200349,-0.422974,1.061538,1.029231,0.936923,0.48,2.44,0.20


** Validation Data 風機狀態正常(0)/異常(1)比例 **

In [6]:
print(valid_desc)
pd.DataFrame(valid_X)[:5]

正常:7680 (0.64 percent), 結冰:4315 (0.36 percent), Total:11995


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25
0,-1.381729,-1.129659,-0.961232,-2.007138,-1.274441,2.679937,0.065687,0.208889,0.217778,0.204444,0.0,0.0,0.0,-1.989,-1.989,-1.96,0.10089,-1.209522,-2.218690,-1.952731,0.629231,0.569231,0.936923,1.28,1.76,0.68
1,-1.182579,-1.166221,-0.939204,-1.908736,-0.197884,2.720707,0.937780,0.208889,0.217778,0.204444,0.0,0.0,0.0,-1.989,-1.989,-1.96,0.10089,0.061109,-2.202909,-1.973765,0.629231,0.569231,0.923077,-0.92,-0.64,0.36
2,-1.267402,-1.159574,-0.947219,-1.744732,-1.128790,2.714883,0.588943,0.208889,0.217778,0.204444,0.0,0.0,0.0,-1.989,-1.989,-1.96,0.10089,-1.209522,-2.202909,-1.952731,0.629231,0.552308,0.923077,-0.72,0.12,-0.52
3,-1.230522,-1.172869,-0.959240,-1.780985,-1.342834,2.726531,0.728478,0.208889,0.217778,0.204444,0.0,0.0,0.0,-1.989,-1.989,-1.96,0.10089,0.061109,-2.202909,-1.952731,0.629231,0.552308,0.923077,-0.60,0.52,-1.20
4,-1.208395,-1.172869,-0.967231,-1.708478,-0.171287,2.720707,0.414524,0.208889,0.217778,0.204444,0.0,0.0,0.0,-1.989,-1.989,-1.96,0.10089,0.061109,-2.218690,-1.952731,0.615385,0.552308,0.923077,0.64,0.24,0.36


In [7]:
print(valid_21_desc)

正常:168930 (0.94 percent), 結冰:10638 (0.06 percent), Total:179568


## 3. Deep Learning

### 3.1 Hyper Parameters

In [8]:
lr = 0.01 # learning rate
n_inputs = 26 # 每一行的维度
n_classes = 2  # RNN最后的输出類別個数

### 3.2 Tensorflow 參數定義

In [9]:
tf.reset_default_graph()
X =tf.placeholder(tf.float32, [None, n_inputs], name="X")
Y_ =tf.placeholder(tf.float32, [None, n_classes], name="Y_")

W = {
    'out_h0': tf.Variable(tf.random_normal([n_inputs,n_classes], stddev=0.01)),
    'wd_h2_1': tf.Variable(tf.random_normal([n_inputs,15], stddev=0.01)),
    'wd_h2_2': tf.Variable(tf.random_normal([15,7], stddev=0.01)),
    'out_h2': tf.Variable(tf.random_normal([7, n_classes]), name="out")
}
#variable_summaries(W['wd1'],'wd1')
#variable_summaries(W['wd2'],'wd2')
#variable_summaries(W['out'],'out')

B = {
    'out_h0': tf.Variable(tf.random_normal([n_classes]), name="out"),
    'bd_h2_1': tf.Variable(tf.random_normal([15])),
    'bd_h2_2': tf.Variable(tf.random_normal([7])),
    'out_h2': tf.Variable(tf.random_normal([n_classes]), name="out"),
}

#variable_summaries(B['bd1'],'bd1')
#variable_summaries(B['bd2'],'bd2')
#variable_summaries(B['out'],'out')

#tfdot() 不看!

### 3.2 網路結構 Network Structure

In [10]:
with tf.name_scope('outlayer_H0'):
    _pred = tf.matmul(X, W['out_h0']) + B['out_h0']
    pred = tf.nn.softmax(_pred, name="pred")
    
with tf.name_scope('accuracy'):
    correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(Y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

### 3.2 誤差函數 Loss Function

In [11]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=_pred, labels=Y_))

### 3.2 Optimizer (gradient descent)

In [12]:
optimizer = tf.train.AdamOptimizer(lr).minimize(loss)

### 3.3 開始訓練  Start Training

In [13]:
init = tf.global_variables_initializer()
# Create a summary to monitor cost tensor
#with tf.name_scope('performance'):  
#    loss_scar = tf.summary.scalar('loss', loss)
    # Create a summary to monitor accuracy tensor
#    acc_scar = tf.summary.scalar('accuracy', accuracy)
    # Merge all summaries into a single op
#merged_summary_op = tf.summary.merge_all()

In [14]:
#先創造一個session, 然後記得要init variable
sess = tf.InteractiveSession()
sess.run(init)

#if not os.path.exists('tb-log_1'):
#    os.mkdir('tb-log_1')
#summary_writer_train = tf.summary.FileWriter('tb-log1/train',graph=tf.get_default_graph())
#summary_writer_validation = tf.summary.FileWriter('tb-log1/validation',graph=tf.get_default_graph())

In [15]:
# training process
epoch = 15
batch_size = 3000
total_batch= len(train_X) / batch_size
for ep in range(epoch+1):
    for i in range(int(total_batch)+1):
        rnd_idx = np.random.choice(train_X.shape[0], batch_size, replace=False)
        batch_x = train_X[rnd_idx]
        batch_y = train_Y[rnd_idx]
        _, acc_v1, loss_v1= sess.run([optimizer, accuracy,loss], feed_dict={X: batch_x, Y_:batch_y})
        #summary_writer_train.add_summary(summary, ep * total_batch + i)
        #if i%100 ==0:
    acc_v2, loss_v2= sess.run([accuracy,loss], feed_dict={X: valid_X , Y_: valid_Y})
    #summary_writer_validation.add_summary(summary, ep * total_batch + i)
    updateProgress('epoch:{x0}, batch:{x4}, train loss:{x1:.3f} acc:{x5:.3f}, test loss:{x3:.3f} acc:{x2:.3f}'.format(x0=ep,x2=round(acc_v2,3),x3=round(loss_v2,3),x4=i,x1=round(loss_v1,3), x5=round(acc_v1,3)))
    print()

epoch:0, batch:19, train loss:0.544 acc:0.728, test loss:0.628 acc:0.650
epoch:1, batch:19, train loss:0.515 acc:0.773, test loss:0.637 acc:0.659
epoch:2, batch:19, train loss:0.465 acc:0.808, test loss:0.582 acc:0.656
epoch:3, batch:19, train loss:0.459 acc:0.800, test loss:0.599 acc:0.665
epoch:4, batch:19, train loss:0.433 acc:0.816, test loss:0.552 acc:0.693
epoch:5, batch:19, train loss:0.428 acc:0.808, test loss:0.564 acc:0.688
epoch:6, batch:19, train loss:0.414 acc:0.816, test loss:0.537 acc:0.720
epoch:7, batch:19, train loss:0.416 acc:0.816, test loss:0.530 acc:0.731
epoch:8, batch:19, train loss:0.400 acc:0.823, test loss:0.536 acc:0.730
epoch:9, batch:19, train loss:0.388 acc:0.827, test loss:0.545 acc:0.724
epoch:10, batch:19, train loss:0.384 acc:0.829, test loss:0.559 acc:0.713
epoch:11, batch:19, train loss:0.389 acc:0.826, test loss:0.570 acc:0.710
epoch:12, batch:19, train loss:0.391 acc:0.821, test loss:0.555 acc:0.726
epoch:13, batch:19, train loss:0.374 acc:0.837, 

## 4. 預測與準確率評估  Evaluation 

#### Predict Training Data

In [16]:
# predict all X and get the accuracy
_ = accuracy.eval({X: train_X , Y_: train_Y})
print('Accuracy:',_)
pred_ = pd.DataFrame(pred.eval({X: train_X}))
train_pred = pred_.apply(np.argmax,axis=1)
pd.Series(train_y).value_counts()
train_pred.value_counts()

print("Precision", precision_score(train_y, train_pred))
print("Recall", sk.metrics.recall_score(train_y, train_pred))

s,_=myscore(train_y, train_pred)
print('score',s,_)

Accuracy: 0.83433


0    35339
1    22179
dtype: int64

0    36066
1    21452
dtype: int64

Precision 0.794844303561
Recall 0.768790297128
score 0.822126833105 {'tn': 30938, 'fp': 4401, 'tp': 17051, 'fn': 5128}


#### Predict  Validataion Data

In [17]:
# predict all X and get the accuracy
_ = accuracy.eval({X: valid_X , Y_: valid_Y})
print('Accuracy:',_)
pred_ = pd.DataFrame(pred.eval({X: valid_X}))
valid_pred = pred_.apply(np.argmax,axis=1)
valid_pred.value_counts()

print("Precision", precision_score(valid_y, valid_pred))
print("Recall", sk.metrics.recall_score(valid_y, valid_pred))

s,_=myscore(valid_y, valid_pred)
print('score',s,_)

Accuracy: 0.710046


1    6015
0    5980
dtype: int64

Precision 0.56957605985
Recall 0.793974507532
score 0.728432566266 {'tn': 5091, 'fp': 2589, 'tp': 3426, 'fn': 889}


** Predict another machine **

In [18]:
# predict all X and get the accuracy
_ = accuracy.eval({X: valid_21_X , Y_: valid_21_Y})
print('Accuracy:',_)
pred_ = pd.DataFrame(pred.eval({X: valid_21_X}))
valid_pred = pred_.apply(np.argmax,axis=1)
valid_pred.value_counts()

print("Precision", precision_score(valid_21_y, valid_pred))
print("Recall", sk.metrics.recall_score(valid_21_y, valid_pred))

s,_=myscore(valid_21_y, valid_pred)
print('score',s,_)

Accuracy: 0.912585


0    164547
1     15021
dtype: int64

Precision 0.331602423274
Recall 0.468227110359
score 0.704397104579 {'tn': 158890, 'fp': 10040, 'tp': 4981, 'fn': 5657}


#### Predict Test Data

In [19]:
pred_ = pd.DataFrame(pred.eval({X: test_X}))
test_pred = pred_.apply(np.argmax,axis=1)
test_pred.value_counts()

0    188235
1     14093
dtype: int64

## 5. 整理預測結果, 正確上傳格式

In [74]:
startTime=0
endTIme=0
search_start=True
search_end=False
abnormal_list=[]
for i,v in enumerate(test_pred):
    if (v==1) & (search_start):
        startTime = test_timeidx[i]
        search_end=True
        search_start=False
    if (v==0) & (search_end):
        endTIme=test_timeidx[i]
        search_start=True
        search_end=False
        abnormal_list.append((startTime,test_timeidx[i-1]))

In [20]:
output_ans=False
if(output_ans):
    with open('test1_08_results.csv','w') as out:
        csv_out=csv.writer(out)
        #csv_out.writerow(['startTime','endTime'])
        for row in abnormal_list:
            csv_out.writerow(row)
